In [1]:
#imports and prep

import cv2
import os
import utils
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import utils
import csv 

In [2]:
def compute_dice_coefficient(path1, path2):
    # Load the masks as  images
    mask1 = cv2.imread(path1)
    mask2 = cv2.imread(path2)

    # Convert the image to grayscale
    mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
    mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to create a binary image
    ret, mask1 = cv2.threshold(mask1, 127, 255, cv2.THRESH_BINARY) 
    ret, mask2 = cv2.threshold(mask2, 127, 255, cv2.THRESH_BINARY) 

   
    # Compute the intersection and sum of the masks
    intersection = np.logical_and(mask1, mask2)
    mask_sum = np.sum(mask1) + np.sum(mask2)

    # Compute the Dice coefficient
    dice_coefficient = (2.0 * (np.sum(intersection)*255)) / mask_sum

    return dice_coefficient

#--- test image paths
#image = "/home/lqmeyers/paintDetect/images/training/f6.3x2022_06_20.mp4.track000353.frame012451.jpg"
#mask_true = "/home/lqmeyers/paintDetect/masks/training/f6.3x2022_06_20.mp4.track000353.frame012451.Paint.png"
#mask_pred = "/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f6.3x2022_06_20.mp4.track000353.frame012451.pred.jpg"

#--- test calling
#dice_coefficient = compute_dice_coefficient(mask_true, mask_pred)
#print("Dice coefficient:", dice_coefficient)
#compare_masks(mask_pred,mask_true,'_',False)

In [5]:
#one function to rule them  all:

def eval_binary_masks(pred_mask_folder,true_mask_folder,image_folder,out_csv,show=True,save=False):
    '''a function to '''
   
    #initialize a folder to store comparison images
    save_dir = pred_mask_folder+'comparisons/'
    os.mkdir(save_dir)

    #initialize csv file to record dice scores:
    with open(out_csv, 'w', newline='') as csvfile:
    # Create a CSV writer object
        writer = csv.writer(csvfile)
        header = ["pred_mask","true_mask","size","mask type","mask size","false_negatives","false_positives","dice_score"]
        writer.writerow(header) 
        #for all files in predicted masks,
        dir_list = os.walk(pred_mask_folder)
        for root, dirs, files in dir_list:
            files = sorted(files)
            print(files)
            #if nested structure, add another walk here for each d in dirs
            for f in files: 
                if f[-4:] != ".png":
                    continue
                true_mask = true_mask_folder+f[:-14]+".Paint.png"
                print("Ground truth file:",true_mask_folder+f[:-14]+".Paint.png")
                pred_mask = root+f
                print("Predicted mask file:",pred_mask)
                
                mask1 = cv2.imread(pred_mask)
                mask2 = cv2.imread(true_mask)
                

                # Convert the image to grayscale
                mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
                mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)

                # Apply a threshold to create a binary image
                ret, mask1 = cv2.threshold(mask1, 127, 255, cv2.THRESH_BINARY) 
                ret, mask2 = cv2.threshold(mask2, 127, 255, cv2.THRESH_BINARY) 
            
                # Ensure that both masks have the same shape
                mask1 = cv2.resize(mask1, (mask2.shape[1], mask2.shape[0]))
                ms = np.count_nonzero(mask2) #add total size of mask

                # Define colors for each overlap condition
                colors = [(0, 0, 0), ( 255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Black, Blue, Green, Red

                # Create an empty image with 3 channels
                result = np.zeros((mask1.shape[0], mask1.shape[1], 3), dtype=np.uint8)

                #initilize pixel counts
                fp = 0
                fn = 0 
                tp = 0 
                tn = 0

                # Compare the masks and assign colors based on the overlap conditions
                for i in range(mask1.shape[0]):
                    for j in range(mask1.shape[1]):
                        if mask1[i, j] == 0 and mask2[i, j] == 0:  # Neither mask has a pixel
                            result[i, j] = colors[0]  # Black
                            tn+=1
                        elif mask1[i, j] > 0 and mask2[i, j] == 0:  # Only mask1 has a pixel
                            result[i, j] = colors[1]  # Blue
                            fp+=1
                        elif mask1[i, j] == 0 and mask2[i, j] > 0:  # Only mask2 has a pixel
                            result[i, j] = colors[2]  # Green
                            fn+=1
                        else:  # Both masks have a pixel
                            result[i, j] = colors[3]  # Red
                            tp+=1
               
                
                
                #open images to combine
                image2 = result
                image_path = image_folder+f[:-14]+".jpg"
                image1 = cv2.imread(image_path)
                print("Image file path: ",image_path)
            
                #change from bgr
                image2 = cv2.cvtColor(image2,cv2.COLOR_BGR2RGB)
                image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2RGB)

                if  np.array(image1).shape != np.array(image2).shape:
                    arr = np.array(image2)
                    arr_reshaped = np.expand_dims(arr, axis=-1)  # Add a new axis at the end
                    arr_reshaped = np.tile(arr_reshaped, (1, 1, 3)) 
                    if  np.array(image1).shape != np.array(arr_reshaped).shape:
                        print('check input image dimensions, remember to put binary masks second')
                        return
                # Stitch the images horizontally
                    stitched_image = np.concatenate((image1, arr_reshaped), axis=1)
                else:
                    stitched_image = np.concatenate((image1, image2), axis=1)

                if save==True:
                    stitched_image = cv2.cvtColor(stitched_image,cv2.COLOR_RGB2BGR) 
                    out_path = save_dir+f[:-14]+".comparison.png"
                    cv2.imwrite(out_path,stitched_image,) #need r'/' after dir when using os to get it
                elif show==True:
                    plt.imshow(stitched_image)
                    plt.axis('off')
                    plt.show()
                
                # Compute the intersection and sum of the masks
                intersection = np.logical_and(mask1, mask2)
                mask_sum = np.sum(mask1) + np.sum(mask2)

                # Compute the Dice coefficient
                dice_coefficient = (2.0 * (np.sum(intersection)*255)) / mask_sum
                writer.writerow([pred_mask,true_mask,mask1.shape,"Binary",ms,fn,fp,dice_coefficient])



In [ ]:
#Run through a folder 
import os

image_folder = "/home/lqmeyers/paintDetect/data/images/testing/"
true_mask_folder = "/home/lqmeyers/paintDetect/data/paint_only_masks/testing/"
pred_mask_folder = "/home/lqmeyers/paintDetect/data/paint_only_masks/predict_2023-07-08-21:41:43/"

eval_binary_masks(pred_mask_folder,true_mask_folder,image_folder,os.path.dirname(pred_mask_folder)+r"/evaluation.csv",save=True,show=True)
    

Making a version of above but for n class segmentation

In [7]:
#supporting functions: 

def fill_in(binary_mask):
    """
    takes in a binary image with a contour that has a hole in it (like a donut)
    and returns a mask where the hole has been filled in (more donut!)
    
    Params: 
    binary_mask(array): numpy array of binary image

    Returns:
    filled_image(array): numpy array of binary image with only one closed contour 
    """
    # Find the contours in the binary image
    contours, hierarchy = cv2.findContours(binary_mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
   
    # Create a blank image to draw the filled contours
    filled_image = np.zeros_like(binary_mask,dtype=np.uint8)
    print(filled_image.shape)

    # Draw filled contours
    cv2.drawContours(filled_image, contours, -1, (255), thickness=-1)

    return(filled_image)

def convert_image_to_classes(image, num_colors):
    """
    Convert the image to numpy array of class ints
    
    Params: 
    image(array): input rgb image as an array
    
    Returns:
    class_image(array): class array of same shape,background = 0
    """

    img_array = np.array(image)
    
    # Reshape the image array to 2D
    reshaped_img = img_array.reshape(-1, 3)
    
    # Get the unique colors from the image
    unique_colors, color_indices = np.unique(reshaped_img, axis=0, return_inverse=True)
    print("number of unique colors found:", unique_colors,"len = ",len(unique_colors))
    
    # Reduce the number of colors if needed
    if len(unique_colors) > num_colors:
       unique_colors = unique_colors[:num_colors]
    
    # Create a lookup table for color mapping
    color_lookup = np.arange(len(unique_colors))
 
    # Map the color indices to the reduced color set
    class_image = color_lookup[color_indices]
    
    # Reshape the class image back to the original shape
    class_image = class_image.reshape(img_array.shape[:2])
    
    return class_image




In [13]:

def eval_multiclass_masks(pred_mask_folder,true_mask_folder,image_folder,num_classes,out_csv,show=True,save=False):
    '''a function to '''
   
    #initialize a folder to store comparison images
    save_dir = pred_mask_folder+'comparisons/'
    os.mkdir(save_dir)

    #initialize csv file to record dice scores:
    with open(out_csv, 'w', newline='') as csvfile:
    # Create a CSV writer object
        writer = csv.writer(csvfile)
        header = ["pred_mask","true_mask","size","mask_type","avg dice_score"]
        writer.writerow(header) 
        #for all files in predicted masks,
        dir_list = os.walk(pred_mask_folder)
        for root, dirs, files in dir_list:
            files = sorted(files)
            print(files)
            #if nested structure, add another walk here for each d in dirs
            for f in files: 
                if f[-4:] != ".png":
                    continue
                true_mask = true_mask_folder+f[:-14]+".png"
                print("Ground truth file:",true_mask_folder+f[:-14]+".png")
                pred_mask = root+f
                print("Predicted mask file:",pred_mask)
                
                mask1 = cv2.imread(pred_mask)
                mask2 = cv2.imread(true_mask)
                
                mask1 = np.asarray(mask1)
                mask2 = np.asarray(mask2)

                mask1 = convert_image_to_classes(mask1,num_classes)
                mask2 = convert_image_to_classes(mask2,num_classes)

                array1 = mask1
                array2 = mask2                

                # Check if the arrays have the same shape
                assert array1.shape == array2.shape, "Arrays must have the same shape"

                # Determine the number of unique classes
                num_classes = max(np.max(array1), np.max(array2)) + 1

                # Create a mask for non-matching pixels
                discrepancy_mask = array1 != array2

                # Create a color map for each class
                colors = []
                #for _ in range(num_classes):
                    #colors.append([random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)])
                colors = [[  128,   128,   128],
                 [128,   0,   0],
                [  0, 128,   0],
                [128, 128,   0],
                [  0,   0, 128]]
                

                # Create the visualization image
                visualization = np.zeros((array1.shape[0], array1.shape[1], 3), dtype=np.uint8)
                for class_id in range(num_classes):
                    visualization[array1 == class_id] = colors[class_id]

                # Set non-matching pixels to black
                visualization[discrepancy_mask] = [0, 0, 0]

                #open images to combine
                image2 = visualization
                image_path = image_folder+f[:-14]+".jpg"
                image1 = cv2.imread(image_path)
                print("Image file path: ",image_path)
            
                #change from bgr
                #image2 = cv2.cvtColor(image2,cv2.COLOR_BGR2RGB)
                image1 = cv2.cvtColor(image1,cv2.COLOR_BGR2RGB)

                if  np.array(image1).shape != np.array(image2).shape:
                    arr = np.array(image2)
                    arr_reshaped = np.expand_dims(arr, axis=-1)  # Add a new axis at the end
                    arr_reshaped = np.tile(arr_reshaped, (1, 1, 3)) 
                    if  np.array(image1).shape != np.array(arr_reshaped).shape:
                        print('check input image dimensions, remember to put binary masks second')
                        return
                # Stitch the images horizontally
                    stitched_image = np.concatenate((image1, arr_reshaped), axis=1)
                else:
                    stitched_image = np.concatenate((image1, image2), axis=1)

                if save==True:
                    stitched_image = cv2.cvtColor(stitched_image,cv2.COLOR_RGB2BGR) 
                    out_path = save_dir+f[:-14]+".comparison.png"
                    cv2.imwrite(out_path,stitched_image,) #need r'/' after dir when using os to get it
                elif show==True:
                    plt.imshow(stitched_image)
                    plt.axis('off')
                    plt.show()
            
                #calc multiclass dice including background
                epsilon = 1e-7
                dice_scores = []
                
                y_true = array2
                y_pred = array1

                for class_id in range(num_classes):
                    #print("calculating class num ",class_id)
                    intersection = np.sum(y_true[y_pred == class_id] == class_id)
                    union = np.sum(y_true == class_id) + np.sum(y_pred == class_id)
                    
                    dice_score = (2.0 * intersection + epsilon) / (union + epsilon)
                    #print("dice =", dice_score)
                    dice_scores.append(dice_score)
                
                average_dice_score = np.mean(dice_scores)
                writer.writerow([pred_mask,true_mask,mask1.shape,"Multiclass n="+str(num_classes),average_dice_score])



In [16]:
import os

image_folder = "/home/lqmeyers/paintDetect/data/images/testing/"
true_mask_folder = "/home/lqmeyers/paintDetect/data/full_masks/testing/"
pred_mask_folder = "/home/lqmeyers/paintDetect/data/full_masks/predict_2023-07-02-02:01:41/"

eval_multiclass_masks(pred_mask_folder,true_mask_folder,image_folder,5,os.path.dirname(pred_mask_folder)+r"/evaluation.csv",save=True,show=True)
    

['evaluation.csv', 'f06x2022_06_28.mp4.track000115.frame001572.full.pred.png', 'f06x2022_06_28.mp4.track000306.frame004886.full.pred.png', 'f06x2022_06_28.mp4.track000344.frame005720.full.pred.png', 'f06x2022_06_28.mp4.track000381.frame007837.full.pred.png', 'f06x2022_06_28.mp4.track000401.frame009363.full.pred.png', 'f06x2022_06_28.mp4.track000401.frame009373.full.pred.png', 'f06x2022_06_28.mp4.track000405.frame009773.full.pred.png', 'f08x2022_06_28.mp4.track000023.frame000404.full.pred.png', 'f08x2022_06_28.mp4.track000066.frame001176.full.pred.png', 'f08x2022_06_28.mp4.track000082.frame001515.full.pred.png', 'f08x2022_06_28.mp4.track000097.frame002154.full.pred.png', 'f09x2022_06_28.mp4.track000153.frame003162.full.pred.png', 'f09x2022_06_28.mp4.track000176.frame005081.full.pred.png', 'f09x2022_06_28.mp4.track000177.frame004381.full.pred.png', 'f09x2022_06_28.mp4.track000213.frame004190.full.pred.png', 'f09x2022_06_28.mp4.track000317.frame005743.full.pred.png', 'f09x2022_06_28.mp4.t

[ WARN:0@26394.333] global loadsave.cpp:244 findDecoder imread_('/home/lqmeyers/paintDetect/data/full_masks/predict_2023-07-02-02:01:41/comparisonsf06x2022_06_28.mp4.track000115.frame001572.comparison.png'): can't open/read file: check file path/integrity
[ WARN:0@26394.333] global loadsave.cpp:244 findDecoder imread_('/home/lqmeyers/paintDetect/data/full_masks/testing/f06x2022_06_28.mp4.track000115.frame001572..png'): can't open/read file: check file path/integrity


ValueError: cannot reshape array of size 1 into shape (3)